# Forest Type Multi-class Classification

#####  Downloaded the dataset from UCI website : https://archive.ics.uci.edu/ml/datasets/covertype
Thought it may be interesting to try and design a ML classification problem and try to predict the forest cover type. There are 4 distinct cover types in this dataset. 

In [2]:
import pandas as pd
import os
from os import path as osp
import tensorflow as tf

/home/tammy/.virtualenvs/vision36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [54]:
folder = '/home/tammy/Downloads/Forest_types'

In [55]:
train = pd.read_csv(osp.join(folder, 'training.csv'))

In [56]:
train.head()
all_data.columns

Index(['class', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9',
       'pred_minus_obs_H_b1', 'pred_minus_obs_H_b2', 'pred_minus_obs_H_b3',
       'pred_minus_obs_H_b4', 'pred_minus_obs_H_b5', 'pred_minus_obs_H_b6',
       'pred_minus_obs_H_b7', 'pred_minus_obs_H_b8', 'pred_minus_obs_H_b9',
       'pred_minus_obs_S_b1', 'pred_minus_obs_S_b2', 'pred_minus_obs_S_b3',
       'pred_minus_obs_S_b4', 'pred_minus_obs_S_b5', 'pred_minus_obs_S_b6',
       'pred_minus_obs_S_b7', 'pred_minus_obs_S_b8', 'pred_minus_obs_S_b9'],
      dtype='object')

In [57]:
test = pd.read_csv(osp.join(folder, 'testing.csv'))
dataset = tf.data.TextLineDataset(osp.join(folder, 'testing.csv'))

In [58]:
all_data = pd.concat([train, test])

In [59]:
cols_to_norm = [x for x in all_data.columns if x!='class']

In [60]:
# normalizing values in all feature columns
all_data[cols_to_norm] = all_data[cols_to_norm].apply(lambda x: ((x-x.min())/x.max()-x.min()))

In [61]:
all_data.head()

,class,b1,b2,b3,b4,b5,b6,b7,b8,b9,...,pred_minus_obs_H_b9,pred_minus_obs_S_b1,pred_minus_obs_S_b2,pred_minus_obs_S_b3,pred_minus_obs_S_b4,pred_minus_obs_S_b5,pred_minus_obs_S_b6,pred_minus_obs_S_b7,pred_minus_obs_S_b8,pred_minus_obs_S_b9
0,d,-30.925234,-22.91875,-46.948980,-53.784884,-42.84,-82.869565,-41.633094,-18.904762,-44.868421,...,64.117161,35.784308,9.355377,11.858575,50.657234,3.755465,9.377208,37.311662,9.285158,11.211258
1,h,-30.504673,-22.95625,-46.948980,-53.662791,-42.92,-82.891304,-41.640288,-18.916667,-44.850877,...,64.127599,36.201462,9.349759,11.882999,51.848723,3.642093,9.377208,37.063706,8.870000,10.830000
2,s,-30.794393,-22.98750,-46.989796,-53.738372,-42.92,-82.927536,-41.697842,-18.916667,-44.885965,...,64.211106,36.269688,9.362600,12.101452,52.412553,4.081047,9.755381,38.001035,9.409593,11.312670
3,s,-30.738318,-22.98125,-46.989796,-53.715116,-42.96,-82.934783,-41.712230,-18.928571,-44.903509,...,64.643257,36.688791,9.303210,11.870787,50.125319,3.540349,9.270609,36.058256,8.969548,10.832567
4,d,-30.757009,-22.83750,-46.903061,-53.715116,-42.79,-82.833333,-41.482014,-18.892857,-44.877193,...,64.056618,35.135185,9.374639,12.104166,49.215745,3.973488,9.549797,37.221744,9.563439,11.470565


In [62]:
numerics = cols_to_norm
for i, col in enumerate(cols_to_norm):
    numerics[i] = (tf.feature_column.numeric_column(col))

b1 = tf.feature_column.numeric_column("b1")
b2 = tf.feature_column.numeric_column("b2")
feat_cols = [b1, b2]

In [63]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn import preprocessing
import warnings

warnings.filterwarnings("ignore") 

#full list of classes
category_classes = all_data['class']
category_classes = category_classes.reshape(category_classes.shape[0],1)
#print(category_classes)
#using a label encoder, and binarizer to convert all unique category_ids to have a column for each class 
le = preprocessing.LabelEncoder() 
lb = preprocessing.LabelBinarizer()

le.fit(all_data['class'])
y_encoded = le.transform(all_data['class'])

lb.fit(y_encoded)
labels = lb.transform(y_encoded)
print(labels)

[[1 0 0 0]
 [0 1 0 0]
 [0 0 0 1]
 ...
 [0 0 0 1]
 [0 0 0 1]
 [0 1 0 0]]


In [68]:
from sklearn.model_selection import train_test_split
x_data = all_data.drop('class', axis=1)

In [69]:
X_train, X_test, y_train, y_test = train_test_split(
...     x_data, labels , test_size=0.33, random_state=42)

X_train = X_train.reset_index(drop=True)

In [87]:
X_train.columns

Index(['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9',
       'pred_minus_obs_H_b1', 'pred_minus_obs_H_b2', 'pred_minus_obs_H_b3',
       'pred_minus_obs_H_b4', 'pred_minus_obs_H_b5', 'pred_minus_obs_H_b6',
       'pred_minus_obs_H_b7', 'pred_minus_obs_H_b8', 'pred_minus_obs_H_b9',
       'pred_minus_obs_S_b1', 'pred_minus_obs_S_b2', 'pred_minus_obs_S_b3',
       'pred_minus_obs_S_b4', 'pred_minus_obs_S_b5', 'pred_minus_obs_S_b6',
       'pred_minus_obs_S_b7', 'pred_minus_obs_S_b8', 'pred_minus_obs_S_b9'],
      dtype='object')

In [72]:
print(X_train.shape, y_train.shape)

(350, 27) (350, 4)


In [91]:
num_labels = 4 # number of classes
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

data = X_train.as_matrix().astype(np.float32)
labels = np.array(y_train).astype(np.float32)

feature_size = data.shape[1]

graph = tf.Graph()

with graph.as_default():
    tf_train_dataset = tf.constant(data)
    tf_train_labels = tf.constant(labels)
Wr(logits=tf.matmul(tf_train_dataset, weights) + biases, 
                                                labels=tf_train_labels))

    optimizer = tf.train.AdamOptimizer(0.1).minimize(loss)
    train_prediction = tf.nn.softmax(logits)

In [92]:
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    for step in range(100001):
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if step % 10000 == 0:
            print('step:{} loss:{:.6f} accuracy: {:.2f}'.format(
                    step, l, accuracy(predictions, labels)))
print(predictions, labels)

step:0 loss:208.791656 accuracy: 15.14
step:10000 loss:4.296745 accuracy: 40.86
step:20000 loss:0.747740 accuracy: 88.00
step:30000 loss:0.763769 accuracy: 89.14
step:40000 loss:0.734552 accuracy: 89.14
step:50000 loss:0.722539 accuracy: 89.43
step:60000 loss:1.053671 accuracy: 85.43
step:70000 loss:4.751954 accuracy: 77.43
step:80000 loss:0.731642 accuracy: 90.86
step:90000 loss:0.747181 accuracy: 90.86
step:100000 loss:0.716342 accuracy: 90.86
[[8.5926259e-11 9.9971443e-01 3.3371955e-10 2.8559662e-04]
 [4.0883716e-14 9.9995828e-01 2.3069710e-12 4.1705374e-05]
 [8.9249788e-08 1.5000348e-05 1.5001363e-07 9.9998474e-01]
 ...
 [9.8140121e-27 0.0000000e+00 1.0000000e+00 0.0000000e+00]
 [3.2468302e-11 9.1154603e-31 1.0000000e+00 5.3647388e-30]
 [7.4006995e-11 9.9894470e-01 4.7564589e-13 1.0552674e-03]] [[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 ...
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]]
